<a href="https://colab.research.google.com/github/Tannongma/SCM.275x/blob/main/SCM_275x_Inventory_Lead_Time.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SCM.275x - Advanced Supply Chain Systems Planning and Network Design
# **Inventory - Lead Time**

### *Before starting, make sure to save a copy of this notebook to your Google Drive!*

## **Initialization**

In [ ]:
# Install necessary packages if they are not already installed

!pip install gurobipy   # Gurobi optimization solver
!pip install pandas     # Pandas for data analysis and manipulation
!pip install folium     # Folium for creating interactive maps
!pip install geopy      # Geopy for computing distances and working with geographic data

!pip install scgraph==2.1.0         # Python package used to compute paths and distances on a real-world transportation network
!pip install scgraph_data==2.0.0    # Python package used to compute paths and distances on a real-world transportation network



In [ ]:
# Import all required packages

# Import all required packages

import pandas as pd                   # For data manipulation and analysis
import gurobipy as grb                # Gurobi optimization library for solving mathematical models
import folium                         # For creating interactive maps
import folium.plugins as plugins      # Additional plugins for folium
from geopy.distance import geodesic   # For calculating geodesic distances between two points

import scgraph                                                  # For computing paths and distances on a real-world transportation network
from scgraph.geographs.us_freeway import us_freeway_geograph    # Data on US highway network (for road paths and distances)
from scgraph.geographs.marnet import marnet_geograph            # Data on maritime routes (for ocean paths and distances)

import scipy.stats as st                                        # Importing the stats module from SciPy for statistical functions (e.g., probability distributions)
import math                                                     # Importing the math module for mathematical functions (e.g., logarithms, square roots)



## **Helper functions**

### **Plotting nodes on a map**

In [ ]:
# Defining a function to plot nodes on a map using folium

def plot_nodes(map,                         # Folium map object to plot the nodes on
               nodes,                       # Dictionary of node objects where each node contains attributes like latitude and longitude
               icon,                        # Icon symbol to use for the markers on the map
               active_color,                # Color of the marker icon for active nodes
               background_color,            # Background color of the marker icon
               inactive_color = 'grey',     # Color of the marker icon for inactive nodes
               ):

    # Loop through each node in the dictionary
    for node in nodes.values():

        # Create a folium marker
        marker = folium.Marker(
            location=[node.lat, node.lon],              # Set the marker's location
            popup = (node.ID + "-" + node.name),        # Create a marker popup with the node ID and name
            icon=plugins.BeautifyIcon(                  # Create a marker's icon
                icon=icon,
                icon_shape="circle",
                text_color=active_color if node.active == True else inactive_color,
                border_color=active_color if node.active == True else inactive_color,
                background_color=background_color,
            )
        )

        # Add a folium marker to the map
        marker.add_to(map)


### **Computing geodesic distance**

In [ ]:
# Defining a function for computing geodesic distances between two locations

def compute_geodesic_distance(origin,       # Origin node object
                              destination,  # Destination node
                              unit='km'):   # Unit ('km' or 'mi'), default value = 'km'

    # Extract coordinates (latitude and longitude) from origin and destination
    origin_coordinates = [origin.lat, origin.lon]
    destination_coordinates = [destination.lat, destination.lon]

    # Compute distance based on the specified unit ('km' or 'mi')
    if unit == 'km':
        distance = geodesic(origin_coordinates, destination_coordinates).km  # Compute distance in kilometers
    elif unit == 'mi':
        distance = geodesic(origin_coordinates, destination_coordinates).mi  # Compute distance in miles

    return distance  # Return the calculated distance


### **Computing the shortest path between two points on a real road or ocean network**

In [ ]:
# Function for computing the shortest path between two points on a real road or ocean network

def shortest_path(origin, destination, mode, result, unit='mi'):

    # Extract coordinates from origin and destination objects
    origin_coordinates = (origin.lat, origin.lon)
    destination_coordinates = (destination.lat, destination.lon)

    # Calculate the shortest path on the ocean network
    if mode == 'ocean':
        output = marnet_geograph.get_shortest_path(
            origin_node={"latitude": origin.lat, "longitude": origin.lon},
            destination_node={"latitude": destination.lat, "longitude": destination.lon},
            output_units= unit
        )

    # Calculate the shortest path on the road network
    elif mode == 'road':
        output = us_freeway_geograph.get_shortest_path(
            origin_node={"latitude": origin.lat, "longitude": origin.lon},
            destination_node={"latitude": destination.lat, "longitude": destination.lon},
            output_units= unit
        )

    # Return the total distance of the path
    if result == 'distance':
        return output['length']

    # Return the coordinates representing the path
    elif result == 'coordinate_path':
        return output['coordinate_path']


### **Plotting flows (real network) on the map**

In [ ]:
def adjustArcPath(path):
    for index in range(1, len(path)):
        x = path[index][1]
        prevX = path[index - 1][1]
        path[index][1] = x - (round((x - prevX)/360,0) * 360)
    return path

def modifyArcPathLong(points, amount):
    return [[i[0], i[1]+amount] for i in points]

def getCleanArcPath(path):
    path = adjustArcPath(path)
    return [
        path,
        modifyArcPathLong(path, 360),
        modifyArcPathLong(path, -360),
        modifyArcPathLong(path, 720),
        modifyArcPathLong(path, -720)
    ]

# Plots real flows on a Folium map based on optimization model results

def plot_real_flows(map,              # Folium map object where flows will be plotted.
               vars,                  # Dictionary of flow decision variables from the optimization model
               nodes,                 # Dictionary of node objects
               width = 10,            # Line width
               opacity = 0.5):        # Opacity of the lines, default is 0.5

    # Iterate over flow decision variables (keys represent node pairs and the transportation mode used)
    for (node1_key, node2_key, t), var in vars.items():

        # Plot only positive flows
        if var.X > 0:

          # If the transportation mode is air only, we add a single polyline to the map
          if t == 'air':

            # Get the coordinates of the nodes for plotting the line
            points = [[nodes[node1_key].lat, nodes[node1_key].lon],
                      [nodes[node2_key].lat, nodes[node2_key].lon]]


            # Add a PolyLine to the map to represent the flow between the nodes
            folium.PolyLine(points,
                            color='magenta',                                # Set the color of the line
                            weight=width,                                   # Set line width
                            opacity=opacity,                                # Set line opacity
                            popup=var.X).add_to(map)                        # Show the flow value in a popup on the map

          # If the transportation mode is a combination of ocean and road, we add two polylines to the map
          elif t == 'ocean_road':

            # Get the shortest path coordinates for the path between the port (i.e., node1_key) and the ocean port (i.e., ocean port associated to node2_key) by ocean
            ocean_path = shortest_path(nodes[node1_key], nodes[node2_key].closest_port, mode = 'ocean', result = 'coordinate_path')

            # Add the road path as a polyline on the map, with width proportional to the flow value
            folium.PolyLine(getCleanArcPath(ocean_path),
                            color='blue',
                            weight=width,
                            opacity=opacity).add_to(map)

            # Get the shortest path coordinates for the path between the closest ocean port (i.e., ocean port associated to node2_key) and the distribution center (i.e., node2_key)
            road_path = shortest_path(nodes[node2_key].closest_port, nodes[node2_key], mode = 'road', result = 'coordinate_path')

            # Add the road path as a polyline on the map, with width proportional to the flow value
            folium.PolyLine(getCleanArcPath(road_path),
                            color='orange',
                            weight=width,
                            opacity=opacity).add_to(map)

### **Finding a closest port**

In [ ]:
# Representing a function for finding the closest port to a given distribution center (DC)
def find_closest_port(dc,             # The DC to consider
                      ports):         # Dictionnary of ports

    distances = dict()  # Store distances between the DC and each port

    for rt in ports.values():  # Iterate over the port objects
        # Compute the geodesic distance between the DC and the current port
        distances[rt.ID] = geodesic((rt.lat, rt.lon), (dc.lat, dc.lon)).mi

    # Identify the port with the minimum distance
    closest_port_id = min(distances.items(), key=lambda x: x[1])[0]
    closest_port = ports[closest_port_id]  # Retrieve the closest port

    return closest_port  # Return the closest port


## **Data setup and preprocessing**

### **Nodes**

#### Reading input files

In [ ]:
# File containing distribution center data
distribution_centers_file = 'https://raw.githubusercontent.com/scm275/problem_sets_scm275/main/inventory_lead_time/distribution_centers.csv'

# Loading distribution center data into a pandas DataFrame
distribution_centers_df = pd.read_csv(distribution_centers_file)

# Displaying the first few rows of the DataFrame to verify the data
distribution_centers_df.head()

,ID,name,country,lat,lon,demand,sigma
0,dc_01,Albany,US,42.656971,-73.794374,300000,15000
1,dc_02,Chicago,US,41.513251,-88.051294,250000,12500
2,dc_03,Dallas,US,32.294702,-96.501628,150000,7500
3,dc_04,Houston,US,29.759714,-94.963986,250000,12500
4,dc_05,Memphis,US,35.133502,-89.966876,250000,12500


In [ ]:
# File containing supplier data
supplier_data_file = 'https://raw.githubusercontent.com/scm275/problem_sets_scm275/main/inventory_lead_time/suppliers.csv'

# Loading supplier data into a pandas DataFrame
suppliers_df = pd.read_csv(supplier_data_file)

# Displaying the first few rows of the DataFrame to verify the data
suppliers_df.head()

,ID,name,country,lat,lon,supply
0,s_01,Haiphong,Vietnam,20.853610,106.577143,2500000
1,s_02,Laem Chabang,Thailand,13.380857,101.024152,1500000
2,s_03,Ningbo,China,29.633147,121.383041,2000000
3,s_04,Shenzhen,China,23.105174,113.785356,2000000


In [ ]:
# File containing port data
ports_data_file = 'https://raw.githubusercontent.com/scm275/problem_sets_scm275/main/inventory_lead_time/ports.csv'

# Loading warehouse data into a pandas DataFrame
ports_df = pd.read_csv(ports_data_file)

# Displaying the first few rows of the DataFrame to verify the data
ports_df.head()

,ID,name,country,dwell_time,lat,lon
0,USBAL,Baltimore,United States,6.0,39.290882,-76.610759
1,USBOS,Boston,United States,6.0,42.355433,-71.060511
2,USHOU,Houston,United States,4.2,29.758938,-95.367697
3,USJAX,Jacksonville,United States,5.8,30.332184,-81.655651
4,USLGB,Long Beach,United States,5.3,33.769016,-118.191604


#### Definition of Classes

In [ ]:
# Class representing a Distribution Center object

class DistributionCenter():
    def __init__(self, ID, name, demand, sigma, lat, lon):
        self.ID = ID              # DC's ID
        self.name = name          # DC's name
        self.demand = demand      # DC's demand
        self.sigma = sigma        # DC's standard deviation of demand
        self.lat = lat            # DC's latitude
        self.lon = lon            # DC's longitude

        self.active = True        # Initializing node as active


In [ ]:
# Class representing a Supplier object

class Supplier():
    def __init__(self, ID, name, lat, lon, supply):
        self.ID = ID            # Supplier's ID
        self.name = name        # Supplier's name
        self.lat = lat          # Supplier's latitude
        self.lon = lon          # Supplier's longitude
        self.supply = supply    # Supplier's available supply

        self.active = True      # Initializing node as active

In [ ]:
# Class representing a Port object

class Port():
    def __init__(self, ID, name, dwell_time, lat, lon):
        self.ID = ID                          # Port's ID
        self.name = name                      # Ports's name
        self.dwell_time = dwell_time          # Ports's dwell time
        self.lat = lat                        # Ports's latitude
        self.lon = lon                        # Ports's longitude

        self.active = True      # Initializing node as active

#### Creating node objects

In [ ]:
nodes = dict()

In [ ]:
# Creating a dictionary of distribution centers objects
distribution_centers = dict()
for i, row in distribution_centers_df.iterrows():
    distribution_centers[row['ID']] = DistributionCenter(ID=row['ID'],             # DC's ID
                                    name=row['name'],             # DC's name
                                    lat=row['lat'],               # DC's latitude
                                    lon=row['lon'],               # DC's longitude
                                    demand=row['demand'],         # DC's demand
                                    sigma=row['sigma'])           # DC's standard deviation of demand

# Merging the distribution centers dictionary into the existing nodes dictionary
nodes = {**nodes, **distribution_centers}

In [ ]:
# Creating a dictionary of supplier objects
suppliers = dict()
for i, row in suppliers_df.iterrows():
    suppliers[row['ID']] = Supplier(ID=row['ID'],           # Supplier's ID
                                    name=row['name'],       # Supplier's name
                                    lat=row['lat'],         # Supplier's latitude
                                    lon=row['lon'],         # Supplier's longitude
                                    supply=row['supply'])   # Supplier's available supply

# Merging the suppliers dictionary into the existing nodes dictionary
nodes = {**nodes, **suppliers}

In [ ]:
# Creating a dictionary of port objects
ports = dict()
for i, row in ports_df.iterrows():
    ports[row['ID']] = Port(ID = row['ID'],                           # Port's ID
                                    name = row['name'],               # Port's name
                                    lat = row['lat'],                 # Port's latitude
                                    lon = row['lon'],                 # Port's longitude
                                    dwell_time = row['dwell_time'])   # Port's dwell time

# Merging the port dictionary into the existing nodes dictionary
nodes = {**nodes, **ports}

#### Finding Closest Port to Distribution Centers

In [ ]:
# Iterate over all distribution center objects in the dictionary

for d in distribution_centers.values():

    # Find the closest port to the current distribution center and assign it to the closest_port attribute of the distribution center
    d.closest_port = find_closest_port(d, ports)


#### Visualizing node objects

In [ ]:
# Create a new map
map = folium.Map([40, 0.0], zoom_start=3)

# Plot distribution center locations with a warehouse icon, green color, and yellow background
plot_nodes(map=map, nodes=distribution_centers, icon='warehouse', active_color='green', background_color='yellow')

# Plot port locations with an anchor icon, blue color, and white background
plot_nodes(map=map, nodes=ports, icon='anchor', active_color='blue', background_color='white')

# Plot supplier locations with an industry icon, orange color, and yellow background
plot_nodes(map=map, nodes=suppliers, icon='industry', active_color='orange', background_color='yellow')

# Add a tile layer for better map visualization (cartodbpositron theme)
folium.TileLayer('cartodbpositron').add_to(map)

# Display the map with all the plotted data
map


### **Arcs**

#### Preprocessing step - distances

In [ ]:
# Creating a dictionary to store distances between suppliers and ports, and ports and customers

distances = dict()  # Initialize an empty dictionary to store distances

# Calculate distances between each supplier and port using the ocean network
for s, supplier in suppliers.items():
    for p, port in ports.items():
        # Store the distance between each supplier-port pair in the dictionary
        distances[s, p] = shortest_path(origin=supplier, destination=port, mode='ocean', result='distance', unit='km')

# Calculate distances between each port and distribution center using the road network
for p, port in ports.items():
    for d, distribution_center in distribution_centers.items():
        # Store the distance between each port-distribution center pair in the dictionary
        distances[p, d] = shortest_path(origin=port, destination=distribution_center, mode='road', result='distance', unit='km')

# Calculate distances between each supplier and distribution center
for s, supplier in suppliers.items():
  for d, distribution_center in distribution_centers.items():
    # Store the distance between each supplier-distribution center pair in the dictionary
        distances[s, d] = compute_geodesic_distance(origin=supplier, destination=distribution_center, unit='km')

#### Arc costs

In [ ]:
# Parameters for computing the ocean and road unit transportation costs

ocean_cost_km_container = 0.16          # Cost per km for ocean transport per container
road_cost_km_container = 1.1            # Cost per km for road transport per container
road_min_cost_container = 1000          # Minimum cost for road transport per container
container_utilization = 55              # Utilization rate of a container (in cubic meters)
unit_volume = 0.0005                    # Volume of the goods per unit (in cubic meters)

# Parameters for computing the airfreight unit cost

air_cost_km_kg = 0.0012                 # Cost per km per kilogram for airfreight
unit_weight = 0.1                       # Weight of the goods per unit (in kilograms)

unit_cost = dict()  # Dictionary to store the unit transportation costs

# Loop through suppliers and distribution centers
for s, supplier in suppliers.items():
  for d, distribution_center in distribution_centers.items():

    # Compute ocean cost from supplier to the closest port to the distribution center (per container)
    ocean_cost_container = distances[s, distribution_center.closest_port.ID] * ocean_cost_km_container

    # Compute road cost from the closest port to the distribution center, considering the minimum cost (per container)
    road_cost_container = max(distances[distribution_center.closest_port.ID, d] * road_cost_km_container, road_min_cost_container)

    # Calculate total ocean + road unit cost for transporting from supplier to distribution center (per unit)
    unit_cost[s, d, 'ocean_road'] = (ocean_cost_container + road_cost_container) / (container_utilization / unit_volume)

    # Calculate airfreight unit cost from supplier to distribution center
    unit_cost[s, d, 'air'] = distances[s, d] * air_cost_km_kg * unit_weight


#### Arc lead times

In [ ]:
# Parameters for computing the lead times in days

ocean_speed = 750  # Speed of ocean transport in kilometers per day
road_speed = 500  # Speed of road transport in kilometers per day
air_speed = 10000  # Speed of air transport in kilometers per day
air_dwell_time = 2  # Dwell time for airfreight at airports in days

lead_time = dict()  # Dictionary to store the lead times for different transportation modes

# Loop through suppliers and distribution centers
for s, supplier in suppliers.items():
  for d, distribution_center in distribution_centers.items():

    # Calculate lead time for ocean + road transport: ocean transport time + road transport time + port dwell time
    lead_time[s, d, 'ocean_road'] = (
      distances[s, distribution_center.closest_port.ID] / ocean_speed  # Ocean transport time
      + distances[distribution_center.closest_port.ID, d] / road_speed  # Road transport time
      + distribution_center.closest_port.dwell_time  # Dwell time at the port
    ) / 365

    # Calculate lead time for air transport: air transport time + airport dwell time
    lead_time[s, d, 'air'] = (
      distances[s, d] / air_speed  # Air transport time
      + air_dwell_time  # Dwell time for airfreight
    ) / 365


### **Transportation modes**

In [ ]:
transportation_modes = ['ocean_road', 'air']

### **Inventory parameters**

In [ ]:
# Additional inventory parameters for safety stock calculation

service_level = 0.95                                # Desired service level (probability of not running out of stock)
safety_stock_factor = st.norm.ppf(service_level)    # Safety stock factor based on the service level, using the inverse of the standard normal distribution

print('Safety Stock factor: ', round(safety_stock_factor, 3))


Safety Stock factor:  1.645


In [ ]:
# Additional inventory parameters for cost calculations

annual_carrying_charge = 0.2                        # Annual carrying charge as a percentage of the unit cost
unit_purchase_cost = 10                             # Purchase cost per unit of the product
ordering_cost = 2500                                # Fixed cost per order


In [ ]:
# Computing and displaying the EOQ at each DC

data = [
    {'DC Name': dc.name, 'Demand': dc.demand,
     'EOQ': round(math.sqrt(2 * ordering_cost * dc.demand / (annual_carrying_charge * unit_purchase_cost)), 2),
     'Order Frequency': round(dc.demand / math.sqrt(2 * ordering_cost * dc.demand / (annual_carrying_charge * unit_purchase_cost)), 1)}
    for dc in distribution_centers.values()
]

df = pd.DataFrame(data)
df

,DC Name,Demand,EOQ,Order Frequency
0,Albany,300000,27386.13,11.0
1,Chicago,250000,25000.00,10.0
2,Dallas,150000,19364.92,7.7
3,Houston,250000,25000.00,10.0
4,Memphis,250000,25000.00,10.0
5,Boston,180000,21213.20,8.5
6,Los Angeles,350000,29580.40,11.8
7,Denver,200000,22360.68,8.9
8,Philadelphia,270000,25980.76,10.4
9,Miami,220000,23452.08,9.4


## **Optimization model**

### **Creating and solving the optimization model**

In [ ]:
# Initialize the Gurobi model
model = grb.Model("Inventory with Lead Times")

# Create decision variables

# Create binary decision variables for each supplier, distribution center, and transportation mode
allocation_dc_s_vars = dict()
for s, supplier in suppliers.items():
    for d, distribution_center in distribution_centers.items():
        for t in transportation_modes:
            allocation_dc_s_vars[s, d, t] = model.addVar(vtype=grb.GRB.BINARY,
                               name="allocation_dc_s_vars_{0}_{1}_{2}".format(s, d, t))

# Create continuous auxiliary variables for each distribution center
aux_vars_h = dict()
for d, distribution_center in distribution_centers.items():
    aux_vars_h[d] = model.addVar(vtype=grb.GRB.CONTINUOUS,
                               name="aux_vars_h{0}".format(d))

# Objective function: calculate the total cost components

# Transportation cost calculation
transportation_cost = grb.quicksum(allocation_dc_s_vars[s, d, t]
                                   * unit_cost[s, d, t]
                                   * distribution_center.demand
                                   for s, supplier in suppliers.items()
                                   for d, distribution_center in distribution_centers.items()
                                   for t in transportation_modes)

# Ordering and holding cost calculation
ordering_holding_cost = math.sqrt(2 * ordering_cost
                                  * unit_purchase_cost
                                  * annual_carrying_charge
                                  * distribution_center.demand)

# Pipeline inventory cost calculation
pipeline_inventory_cost = grb.quicksum(annual_carrying_charge
                                       * unit_purchase_cost
                                       * allocation_dc_s_vars[s, d, t]
                                       * distribution_center.demand
                                       * lead_time[s, d, t]
                                   for s, supplier in suppliers.items()
                                   for d, distribution_center in distribution_centers.items()
                                   for t in transportation_modes)

# Safety stock cost calculation
safety_stock_cost = grb.quicksum(safety_stock_factor
                                 * annual_carrying_charge
                                 * unit_purchase_cost
                                 * aux_vars_h[d]
                                 * distribution_center.sigma
                                 for d, distribution_center in distribution_centers.items())

# Set the objective function to minimize the total cost
total_cost = transportation_cost + ordering_holding_cost + pipeline_inventory_cost + safety_stock_cost
model.setObjective(total_cost, grb.GRB.MINIMIZE)

# Adding constraints

# Ensure the total allocation for each supplier does not exceed its supply capacity
for s, supplier in suppliers.items():
    model.addConstr(grb.quicksum(allocation_dc_s_vars[s, d, t]
                                 * distribution_centers[d].demand
                                 for d, distribution_center in distribution_centers.items()
                                 for t in transportation_modes) <= supplier.supply)

# Ensure that each distribution center is allocated to exactly one supplier and transportation mode
for d, distribution_center in distribution_centers.items():
    model.addConstr(grb.quicksum(allocation_dc_s_vars[s, d, t]
                                 for s, supplier in suppliers.items()
                                 for t in transportation_modes) == 1)

# Ensure auxiliary variable relationships for each distribution center
for d, distribution_center in distribution_centers.items():
    model.addConstr(aux_vars_h[d]**2 == grb.quicksum(allocation_dc_s_vars[s, d, t]
                                                      * lead_time[s, d, t]
                                                      for s, supplier in suppliers.items()
                                                      for t in transportation_modes))

# Solve the optimization model
model.optimize()


Restricted license - for non-production use only - expires 2025-11-24
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 24 rows, 180 columns and 320 nonzeros
Model fingerprint: 0x381345c6
Model has 20 quadratic constraints
Variable types: 20 continuous, 160 integer (160 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+05]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [8e-03, 1e-01]
  Objective range  [2e+04, 6e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+06]
Presolve time: 0.00s
Presolved: 84 rows, 180 columns, 840 nonzeros
Presolved model has 20 bilinear constraint(s)

Solving non-convex MIQCP

Variable types: 20 continuous, 160 integer (160 binary)

Root relaxation: objective 1.083636e+06, 45 iterations, 0.00 seconds (0.00 work units)

 

In [ ]:
#Printing costs as portion of total cost

print('Safety Stock Cost: ', round(safety_stock_cost.getValue() / total_cost.getValue()*100, 2))
print('Transportation Cost', round(transportation_cost.getValue() / total_cost.getValue()*100,2))
print('Pipeline Inventory Cost', round(pipeline_inventory_cost.getValue() / total_cost.getValue() * 100, 2))
print('Ordering and Holding Cost', round(ordering_holding_cost / total_cost.getValue() * 100, 2))

print('')

#Printing average lead time of each DC
for d, dc in distribution_centers.items():
  print (dc.name, round((aux_vars_h[d].X)**2 * 365,2))


Safety Stock Cost:  17.42
Transportation Cost 14.3
Pipeline Inventory Cost 64.45
Ordering and Holding Cost 3.83

Albany 30.71
Chicago 33.68
Dallas 32.07
Houston 29.73
Memphis 31.75
Boston 30.63
Los Angeles 21.56
Denver 23.05
Philadelphia 31.55
Miami 30.57
Seattle 18.32
San Francisco 22.8
Atlanta 31.83
Orlando 33.42
Phoenix 20.92
Las Vegas 20.62
Minneapolis 34.98
Kansas City 33.99
San Antonio 30.28
New Orleans 30.77


## **Solution visualization and analysis**

In [ ]:
# Create a new map
map = folium.Map([40, 0.0], zoom_start=3)

# Plot distribution center locations with a warehouse icon, green color, and yellow background
plot_nodes(map=map, nodes=distribution_centers, icon='warehouse', active_color='green', background_color='yellow')

# Plot port locations with an anchor icon, blue color, and white background
plot_nodes(map=map, nodes=ports, icon='anchor', active_color='blue', background_color='white')

# Plot supplier locations with an industry icon, orange color, and yellow background
plot_nodes(map=map, nodes=suppliers, icon='industry', active_color='orange', background_color='yellow')

# Plot the flows with a maximum line width of 20
plot_real_flows(map=map, width=10, vars=allocation_dc_s_vars, nodes = nodes)

# Add a tile layer for better map visualization (cartodbpositron theme)
folium.TileLayer('cartodbpositron').add_to(map)

# Display the map with all the plotted data
map

